In [ ]:
import numpy as np
import copy
from numpy import linalg as LA
import scipy as sp
from scipy.linalg import block_diag
#from group_lasso import GroupLasso
from sklearn.linear_model import LinearRegression
from scipy.linalg import toeplitz
import time
from numba import jit
import os
import subprocess
import matlab.engine
from group_lasso import GroupLasso
import gurobipy as gp
from gurobipy import GRB
import picos as pic
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
from sklearn.linear_model import Ridge

In [ ]:
# generate syn data https://arxiv.org/pdf/1001.0736.pdf section 4

def Toeplitz_CovMat(pho, m):
    CovMat = np.zeros((m,m))
    for i in range(m):
        for j in range(i,m):
            CovMat[i,j] = np.power(pho, j-i)
            CovMat[j,i] = CovMat[i,j]
    return CovMat


def SynData(n, p, b, alpha, pho):
    #n = int(alpha*21*np.log(p-21))
    #print("n:", n)
    Sparse = [8, 6, 4, 2, 1]
    Group = {}
    for i in range(len(Sparse)):
        Group[i] = 10*i + np.random.permutation(b)[0:Sparse[i]]
    #print(Group)
    cov_single = 0.2*np.ones((b,b))#Toeplitz_CovMat(pho, b)#0.01*np.ones((b,b))
    np.fill_diagonal(cov_single, 1)
    #print(cov_single)
    Cov = cov_single
    for i in range(int(p/b) - 1):
        Cov = block_diag(Cov, cov_single)
    Mean = np.zeros(p)
    #Cov[Cov==0] = 0.1
    #w, _ = LA.eig(Cov)
    #print(Cov.shape)
    #print(Cov)
    #print(w)
    
    X = np.random.multivariate_normal(Mean, Cov, n)
    #print(X.shape)
    w = np.zeros(p)
    for i in range(len(Group)):
        RandBin = 2*np.random.randint(0, 2, size=len(Group[i]))-1
        w[Group[i]] = 1/np.sqrt(21)#RandBin
    w = w[:,np.newaxis]
    #print(w)
    Noise = np.random.normal(0, 0.1, n)
    Noise = Noise[:,np.newaxis]

    #print(w.shape)
    #print(Noise.shape)

    X = X
    SNR_Expect = 6
    Noise = (np.linalg.norm(X.dot(w)) / np.sqrt(SNR_Expect)) * (Noise / np.linalg.norm(Noise))

    SNR = np.linalg.norm(X.dot(w))/np.linalg.norm(Noise)

    #print(SNR**2)

    y = X.dot(w) + Noise
    
     # groups info for L0_GL
    Group_L0GL = {}
    for i in range(10):
        Group_L0GL[i] = range(i*10, (i+1)*10)
    #print(Group0)

    # groups info for L1_GL
    Group_L1GL = np.zeros((100,1))
    for key in Group_L0GL:
        Group_L1GL[Group_L0GL[key],0] = key
    #print(Group_L1GL.T)
    
    return X, y, w, Group_L0GL, Group_L1GL

def SynData_iid(n, p, b):
    #n = int(alpha*21*np.log(p-21))
    #print("n:", n)
    Sparse = [10,10,10,10,10]#[8, 6, 4, 2, 1]
    Group = {}
    for i in range(len(Sparse)):
        Group[i] = 100*i + np.random.permutation(int(p/b))[0:Sparse[i]]
    print(Group)
    #cov_single = 0.2*np.ones((b,b))#Toeplitz_CovMat(pho, b)#0.01*np.ones((b,b))
    #np.fill_diagonal(cov_single, 1)
    #print(cov_single)
    #Cov = cov_single
    #for i in range(int(p/b) - 1):
    #    Cov = block_diag(Cov, cov_single)
    Mean = np.zeros(p)
    Cov = np.eye(p)
    #Cov[Cov==0] = 0.1
    #w, _ = LA.eig(Cov)
    #print(Cov.shape)
    #print(Cov)
    #print(w)
    
    X = np.random.multivariate_normal(Mean, Cov, n)
    #print(X.shape)
    w = np.zeros(p)
    for i in range(len(Group)):
        RandBin = 2*np.random.randint(0, 2, size=len(Group[i]))-1
        w[Group[i]] = RandBin
    w = w[:,np.newaxis]
    #print(w)
    Noise = np.random.normal(0, 0.1, n)
    Noise = Noise[:,np.newaxis]

    #print(w.shape)
    #print(Noise.shape)

    X = X
    SNR_Expect = 4
    Noise = (np.linalg.norm(X.dot(w)) / np.sqrt(SNR_Expect)) * (Noise / np.linalg.norm(Noise))

    SNR = np.linalg.norm(X.dot(w))/np.linalg.norm(Noise)

    #print(SNR**2)

    y = X.dot(w) + Noise
    
     # groups info for L0_GL
    Group_L0GL = {}
    for i in range(10):
        Group_L0GL[i] = range(i*100, (i+1)*100)
    #print(Group0)

    # groups info for L1_GL
    Group_L1GL = np.zeros((p,1))
    for key in Group_L0GL:
        Group_L1GL[Group_L0GL[key],0] = key
    #print(Group_L1GL.T)
    
    return X, y, w, Group_L0GL, Group_L1GL

def Generate_RotationMat(u,k):
    u_ = u / np.linalg.norm(u)
    At = np.random.rand(k,k-1)
    A = np.hstack([u_, At])
    while np.linalg.matrix_rank(A)<k:
        At = np.random.rand(k,k-1)
        A = np.hstack([u_, At])
    Q,R = np.linalg.qr(A)
    P = np.hstack([u_, Q[:,1:]])
    Q_ = np.random.rand(k-1,k-1)
    while np.linalg.matrix_rank(Q_)<k-1:
        Q_ = np.random.rand(k-1,k-1)
    QM = P.dot(block_diag(1, Q_)).dot(P.T)
    return QM
    

def SynDataRotation(k, h, zeta, n, d, B, r):
    #k is the number of nonzero features
    #h is the number of groups that contain the k nonzero features in w1
    #zeta: zeta * h is the number of groups that contain the k nonzero features in w2
    #n is the number of samples
    #d is the number of feature dimension including zero and nonzero features
    #B is the total number of groups including contributing group and non-contributing group d/B should be integer
    #r is the N(0,r^2) noise
    #example: d=500, B=50, h=10, zeta=3, k=60
    if h*d < B*k:
        print("Error: h*d should be larger than B*k")
    
    # generate u, the true feature
    u = 2*(2*np.random.rand(k,1)-1)
    # generate the Rotation matrix, s.t. Qu = u
    Q = Generate_RotationMat(u,k)
    # generate X1 size: n by k
    X1 = np.random.multivariate_normal(np.zeros(k), np.eye(k), n)
    # generate X2 size: n by k
    X2 = X1.dot(Q)
    # generate X3 size: n by d-2k
    X3 = np.random.multivariate_normal(np.zeros(d-2*k), np.eye(d-2*k), n)
    
    # distribute the clomun of X1 on the first h groups
    n_group = int(d/B)
    k_group = int(k/h)
    IndX1 = list()
    for i in range(h):
        indt = np.random.permutation(n_group)[0:k_group]
        indt = np.sort(indt)
        IndX1.extend((i*n_group)+indt)
    
    # distribute the clomun of X1 on the first zeta*h groups after the first h groups
    k_group_ = int(k/(zeta*h))
    IndX2 = list()
    for i in range(zeta*h):
        indt = np.random.permutation(n_group)[0:k_group_]
        indt = np.sort(indt)
        IndX2.extend((h*n_group+i*n_group)+indt)
        
    IndX3 = list(set(range(d))-set(IndX1+IndX2))
    
    X = np.zeros((n,d))
    X[:,IndX1] = X1
    X[:,IndX2] = X2
    X[:,IndX3] = X3
    
    w1 = np.zeros((d,1))
    w1[IndX1] = u
    w2 = np.zeros((d,1))
    w2[IndX2] = u
    
    Noise = np.random.normal(0, r, n)
    Noise = Noise[:,np.newaxis]
    
    SNR_Expect = 6
    Noise = (np.linalg.norm(X.dot(w1)) / np.sqrt(SNR_Expect)) * (Noise / np.linalg.norm(Noise))

    SNR = np.linalg.norm(X.dot(w1))/np.linalg.norm(Noise)

    #print(SNR**2)
    
    y = (X.dot(w1)) + Noise
    
    # groups info for L0_GL
    Group_L0GL = {}
    for i in range(B):
        Group_L0GL[i] = range(i*n_group, (i+1)*n_group)
    #print(Group0)

    # groups info for L1_GL
    Group_L1GL = np.zeros((d,1))
    for key in Group_L0GL:
        Group_L1GL[Group_L0GL[key],0] = key
    #print(Group_L1GL.T)
    
    return X, y, w1, w2, Group_L0GL, Group_L1GL

def CheckGroup(IndPred, Group_L0GL):
    SelectedGroup = list()
    for ind in IndPred:
        for gid in range(len(Group_L0GL)):
            if ind in Group_L0GL[gid]:
                SelectedGroup.extend([gid])
    SelectedGroup = list(set(SelectedGroup))
    return SelectedGroup
    

In [ ]:
def Data4Matlab(X,w,y):
    d = len(w)
    n = len(y)
    # for matlab
    Folder = "./RandomEnsembleI_Known_k_h/"
    fn1 = Folder + "Xnd.txt"
    fh1 = open(fn1,'w')
    for i in range(n):
        for j in range(d-1):
            fh1.write("%f\t" % (X[i,j]))
        fh1.write("%f\n" % (X[i,d-1]))
    fh1.close()

    fn2 = Folder + "yn.txt"
    fh2 = open(fn2, 'w')
    for i in range(n):
        fh2.write("%f\n"%(y[i]))
    fh2.close()

    fn3 = Folder + "wd.txt"
    fh3 = open(fn3, 'w')
    for i in range(d):
        fh3.write("%f\n"%(w[i]))
    fh3.close()

def DataPartition(X, y, ratio):
    n = y.size
    ntrain = int(n*ratio)
    ntest = n - ntrain
    
    RandInd = np.random.permutation(n)
    Indtrain = RandInd[0:ntrain]
    Indtest = RandInd[ntrain:]
    
    Xtrain = X[Indtrain,:]
    ytrain = y[Indtrain,:]
    Xtest = X[Indtest,:]
    ytest = y[Indtest,:]
    
    return Xtrain, ytrain, Xtest, ytest

def OutSample_MSE(X, y, w):
    MSE = np.linalg.norm(X.dot(w)-y)**2
    return MSE

In [ ]:
n=40
p=1000
b=10
B = 10
alpha = 1.8
d = p
pho = 0.2
nc = range(100,1601,100)
ratio = 0.8
htrue = range(5)
NumRun = 3

h = 5
k = 50

X, y, w, Group_L0GL, Group_L1GL  = SynData_iid(500, p, b)
Data4Matlab(X,w,y)

## Random Ensemble I known h and k

In [ ]:
L0GL_IAcc = np.zeros((len(nc), NumRun))
L0GL_GAcc = np.zeros((len(nc), NumRun))
L0GL_Time = np.zeros((len(nc), NumRun))

for i in range(len(nc)):
    n = nc[i]
    
    #X, y, w1, w2, Group_L0GL, Group_L1GL = SynDataRotation(k, h, zeta, n, d, B, r)
    #w = w1
    X, y, w, Group_L0GL, Group_L1GL  = SynData_iid(n, p, b)
    w1 = w

    for j in range(NumRun):
        X, y, w, Group_L0GL, Group_L1GL  = SynData_iid(n, p, b)
        w1 = w
        # for matlab
        Data4Matlab(X,w,y)
       
        # L0GL
        pho = np.sqrt(len(y))
        eng = matlab.engine.start_matlab()
        Tstart = time.time()
        ret = eng.L0GLPQN_Sim1_Known_kh_new(float(pho), int(k), int(h))#nargout=0
        Time = time.time()-Tstart
        eng.quit()
        L0GL_Time[i,j] = Time
        
        zout = np.zeros(b)
        uout = np.ravel(np.array(ret))
        for ii in range(b):
            zout[ii] = np.max(uout[Group_L0GL[ii]])
        zht = np.argsort(-zout)[0:h]
        zind = list()
        for ii in zht:
            zind.extend(Group_L0GL[ii])
        Indt = np.argsort(-uout[zind])[0:k]
        Indn0 = [zind[i] for i in Indt]
        
        IndT = np.nonzero(w)[0]
        OverlapInd = list(set(IndT) & set(Indn0))
        L0GL_IAcc[i,j] = len(OverlapInd) /k

        SGroup = CheckGroup(Indn0, Group_L0GL)
        TGroup = range(h)
        OverlapG = list(set(SGroup) & set(TGroup))
        L0GL_GAcc[i,j] = len(OverlapG) / h  

            

In [ ]:
#output to txt file
def output_txt(Fn, Mat):
    nr, nc = Mat.shape
    fh = open(Fn, 'w')
    for i in range(nr):
        for j in range(nc):
            fh.write("%f\t"%(Mat[i,j]))
        fh.write("\n")
    fh.close()

output_txt("./RandomEnsembleI_Known_k_h/REI_AccI_L0GL_SNR4.txt", L0GL_IAcc)
output_txt("./RandomEnsembleI_Known_k_h/REI_AccG_L0GL_SNR4.txt", L0GL_GAcc)
output_txt("./RandomEnsembleI_Known_k_h/REI_Time_L0GL_SNR4.txt", L0GL_Time)
